In [2]:
from sklearn.model_selection import train_test_split
import json
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import keras
from keras import layers
import pandas as pd
import pickle

Using TensorFlow backend.
/home/maciej/anaconda3/envs/eshop-prediction/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maciej/anaconda3/envs/eshop-prediction/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maciej/anaconda3/envs/eshop-prediction/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.in

In [3]:
with open('records-merged.json') as file:
    sessions = json.load(file)

with open('buys2.json') as file:
    buys = json.load(file)


In [4]:
features = pd.DataFrame(sessions)
labels = pd.DataFrame(buys)

# split to train and test
RANDOM_STATE = 55
TEST_SIZE = 0.25

X_train, X_test, y_train, y_test = train_test_split(features, labels,
                                                    test_size=TEST_SIZE,
                                                    shuffle=False,
                                                    random_state=RANDOM_STATE)

In [5]:
# FIT TRANSFORM
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


attrs_count = X_train.shape[1]

In [6]:
from sklearn.model_selection import GridSearchCV

def create_model(units1, units2):
    model = keras.Sequential()
    model.add(layers.Dense(units1, activation='relu', input_shape=(attrs_count,)))
    model.add(layers.Dense(units2, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    return model

from keras.wrappers.scikit_learn import KerasClassifier
kerasClf = KerasClassifier(build_fn=create_model,batch_size=25)

# Set the parameters by cross-validation
tuned_parameters = [{
    'units1': [100,120],
    'units2': [10,20,30],
    }]


clf = GridSearchCV(kerasClf, tuned_parameters, cv=5)
clf.fit(X_train, y_train)

print("Best Hyperparameters found are:")
print(clf.best_params_)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/1
6969/6969 [==============================] - 1s 73us/step
Epoch 1/1
6969/6969 [==============================] - 0s 71us/step
Epoch 1/1
6968/6968 [==============================] - 0s 44us/step

In [7]:
y_pred = clf.predict(X_test)
y_pred = [x[0] for x in y_pred]

In [8]:
print("Balanced accuracy score: ")
print(balanced_accuracy_score(y_test.values.tolist(), y_pred))

Balanced accuracy score: 
0.6063540133578256


In [9]:
print(confusion_matrix(y_test, y_pred))
print(confusion_matrix(y_test, [0 for _ in range(len(y_test))]))

[[11133    49]
 [  339    94]]
[[11182     0]
 [  433     0]]


In [10]:
par = clf.best_params_
model = create_model(par['units1'], par['units2'])
model.fit(X_train, y_train)

Epoch 1/1
34842/34842 [==============================] - 6s 184us/step - loss: 0.1183 - acc: 0.9637

In [ ]:
with open ('../5-server/core/model.pckl', 'wb') as file:
    pickle.dump(model,file)

with open ('../5-server/core/scaler.pckl', 'wb') as file:
    pickle.dump(sc,file)

In [11]:
with open ('model.pckl', 'wb') as file:
    pickle.dump(model,file)